In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 0. Instalar pacotes e importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

# 1. Leitura das bases

In [ ]:
# Treino
train = pd.read_csv('/content/drive/MyDrive/Prevencao_fraude/train.csv')

# Teste
test = pd.read_csv('/content/drive/MyDrive/Prevencao_fraude/test.csv')

In [ ]:
train.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD
0,59383,2021-08-01 00:04:37,323,217,4.60,0
1,59384,2021-08-01 00:12:10,6,429,8.61,0
2,59385,2021-08-01 00:12:34,714,1011,64.00,0
3,59386,2021-08-01 00:15:40,266,1969,12.72,0
4,59387,2021-08-01 00:16:01,890,1482,98.88,0


In [ ]:
terminal = pd.read_csv('/content/drive/MyDrive/Prevencao_fraude/terminal.csv')
terminal.head()

,TERMINAL_ID,x_terminal_id,y_terminal_id
0,0,9.388886,44.298820
1,1,12.204779,38.423219
2,2,16.123444,41.642938
3,3,13.341542,37.858452
4,4,11.964165,55.351051


In [ ]:
customer = pd.read_csv('/content/drive/MyDrive/Prevencao_fraude/customer.csv')
customer.head()

,CUSTOMER_ID,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day
0,0,10.950170,59.768684,62.262521,31.131260,2.179533
1,1,13.671851,52.775318,46.570785,23.285393,3.567092
2,2,-9.381829,38.617619,80.213879,40.106939,2.115580
3,3,-3.745116,40.551744,11.748426,5.874213,0.348517
4,4,4.312412,51.067100,78.924891,39.462446,3.480049


In [ ]:
# Treino
df_merged = pd.merge(train, customer, on='CUSTOMER_ID', how='left')
df_train_merged = pd.merge(df_merged, terminal, on='TERMINAL_ID', how='left')

# Teste
df_merged = pd.merge(test, customer, on='CUSTOMER_ID', how='left')
df_test_merged = pd.merge(df_merged, terminal, on='TERMINAL_ID', how='left')

# Convertendo a coluna TX_DATETIME para datetime
df_train_merged['TX_DATETIME'] = pd.to_datetime(df_train_merged['TX_DATETIME'])
df_test_merged['TX_DATETIME'] = pd.to_datetime(df_test_merged['TX_DATETIME'])

# Criando a variável que indica o mês de referência
df_train_merged['month_reference'] = df_train_merged['TX_DATETIME'].dt.strftime('%Y%m')
df_test_merged['month_reference'] = df_test_merged['TX_DATETIME'].dt.strftime('%Y%m')

df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,x_terminal_id,y_terminal_id,month_reference
0,59383,2021-08-01 00:04:37,323,217,4.60,0,-0.362432,39.512573,7.353061,3.676530,3.324124,2.439994,50.324767,202108
1,59384,2021-08-01 00:12:10,6,429,8.61,0,-0.466155,38.210214,18.618562,9.309281,3.778676,11.450431,49.902275,202108
2,59385,2021-08-01 00:12:34,714,1011,64.00,0,-0.682848,38.095822,82.620413,41.310207,3.723765,13.616103,46.150744,202108
3,59386,2021-08-01 00:15:40,266,1969,12.72,0,14.309093,40.607318,9.852171,4.926085,3.862067,12.286148,45.514582,202108
4,59387,2021-08-01 00:16:01,890,1482,98.88,0,-4.209270,36.454733,83.660035,41.830018,3.128315,5.168861,51.234704,202108


In [ ]:
df_train_merged.shape

(291231, 14)

In [ ]:
df_train_merged.describe()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,x_terminal_id,y_terminal_id
count,291231.000000,291231,291231.000000,291231.000000,291231.000000,291231.000000,291231.000000,291231.000000,291231.000000,291231.000000,291231.000000,291231.000000,291231.000000
mean,204998.000000,2021-10-16 02:06:44.044813056,496.256202,993.718526,53.182274,0.022601,6.369599,46.715810,51.710394,25.855197,2.650934,6.929841,46.906274
min,59383.000000,2021-08-01 00:04:37,0.000000,0.000000,0.000000,0.000000,-15.722968,36.442978,5.063101,2.531550,0.018782,-16.913020,34.480700
25%,132190.500000,2021-09-08 05:17:43.500000,249.000000,496.000000,21.130000,0.000000,-0.691936,40.348956,28.606102,14.303051,1.956902,0.274440,40.835133
50%,204998.000000,2021-10-16 05:55:26,498.000000,1000.000000,44.770000,0.000000,8.429219,48.108743,51.641807,25.820903,2.836935,8.534868,47.972450
75%,277805.500000,2021-11-23 06:15:18.500000,741.000000,1483.000000,76.850000,0.000000,12.479186,51.567670,75.288972,37.644486,3.438147,12.748856,51.636076
max,350613.000000,2021-12-30 23:58:21,999.000000,1999.000000,291.150000,1.000000,25.192465,60.406826,99.981815,49.990907,3.999725,26.843163,64.060717
std,84071.292461,NaN,285.274966,575.267551,39.573329,0.148627,8.551536,6.486223,27.475610,13.737805,0.961907,8.708658,6.408352


# 2. Feature Engineering

## 2.1 Indicadores de tempo

In [ ]:
# Função para classificar o período do dia
def classify_period(hour):
    if 5 <= hour < 12:
        return 'manhã'
    elif 12 <= hour < 18:
        return 'tarde'
    else:
        return 'noite'

# Aplicar a função para criar a nova coluna 'PERIODO_DIA'
df_train_merged['PERIODO_DIA'] = df_train_merged['TX_DATETIME'].dt.hour.apply(classify_period)
df_test_merged['PERIODO_DIA'] = df_test_merged['TX_DATETIME'].dt.hour.apply(classify_period)

df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,x_terminal_id,y_terminal_id,month_reference,PERIODO_DIA
0,59383,2021-08-01 00:04:37,323,217,4.60,0,-0.362432,39.512573,7.353061,3.676530,3.324124,2.439994,50.324767,202108,noite
1,59384,2021-08-01 00:12:10,6,429,8.61,0,-0.466155,38.210214,18.618562,9.309281,3.778676,11.450431,49.902275,202108,noite
2,59385,2021-08-01 00:12:34,714,1011,64.00,0,-0.682848,38.095822,82.620413,41.310207,3.723765,13.616103,46.150744,202108,noite
3,59386,2021-08-01 00:15:40,266,1969,12.72,0,14.309093,40.607318,9.852171,4.926085,3.862067,12.286148,45.514582,202108,noite
4,59387,2021-08-01 00:16:01,890,1482,98.88,0,-4.209270,36.454733,83.660035,41.830018,3.128315,5.168861,51.234704,202108,noite


In [ ]:
# Extração de features temporais para o DataFrame de treino (df_train_merged)
df_train_merged['hour_of_day'] = df_train_merged['TX_DATETIME'].dt.hour          # Hora do dia
df_train_merged['day_of_week'] = df_train_merged['TX_DATETIME'].dt.dayofweek     # Dia da semana (0=segunda-feira, 6=domingo)
df_train_merged['day_of_month'] = df_train_merged['TX_DATETIME'].dt.day          # Dia do mês
df_train_merged['month'] = df_train_merged['TX_DATETIME'].dt.month               # Mês
df_train_merged['year'] = df_train_merged['TX_DATETIME'].dt.year                 # Ano
df_train_merged['is_weekend'] = df_train_merged['TX_DATETIME'].dt.dayofweek >= 5 # Flag para finais de semana (sábado e domingo)

# # Extração de features temporais para o DataFrame de teste (df_test_merged)
df_test_merged['hour_of_day'] = df_test_merged['TX_DATETIME'].dt.hour          # Hora do dia
df_test_merged['day_of_week'] = df_test_merged['TX_DATETIME'].dt.dayofweek     # Dia da semana (0=segunda-feira, 6=domingo)
df_test_merged['day_of_month'] = df_test_merged['TX_DATETIME'].dt.day          # Dia do mês
df_test_merged['month'] = df_test_merged['TX_DATETIME'].dt.month               # Mês
df_test_merged['year'] = df_test_merged['TX_DATETIME'].dt.year                 # Ano
df_test_merged['is_weekend'] = df_test_merged['TX_DATETIME'].dt.dayofweek >= 5 # Flag para finais de semana (sábado e domingo)

## 2.2 Diferença de tempo entre duas transações consecutivas para o mesmo cliente

In [ ]:
# Ordenar por CUSTOMER_ID e TX_DATETIME
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Calcular a diferença de tempo entre transações consecutivas do mesmo cliente
df_train_merged['time_diff_transactions'] = df_train_merged.groupby('CUSTOMER_ID')['TX_DATETIME'].diff()

# Ordenar por CUSTOMER_ID e TX_DATETIME
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Calcular a diferença de tempo entre transações consecutivas do mesmo cliente
df_test_merged['time_diff_transactions'] = df_test_merged.groupby('CUSTOMER_ID')['TX_DATETIME'].diff()

df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,y_terminal_id,month_reference,PERIODO_DIA,hour_of_day,day_of_week,day_of_month,month,year,is_weekend,time_diff_transactions
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,59.061828,202108,noite,3,6,1,8,2021,True,NaT
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,37.452541,202108,manhã,5,6,1,8,2021,True,0 days 02:29:38
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,57.292877,202108,manhã,10,6,1,8,2021,True,0 days 05:09:34
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,61.073466,202108,tarde,15,6,1,8,2021,True,0 days 04:58:13
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,39.053767,202108,noite,19,6,1,8,2021,True,0 days 03:47:05


## 2.3 Distância do cliente ao terminal

In [ ]:
# Função para calcular a distância entre duas coordenadas
import math

def haversine(lon1, lat1, lon2, lat2):
    # Converter de graus para radianos
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Diferenças das coordenadas
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Fórmula de Haversine
    a = math.sin(delta_lat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Raio da Terra em km
    R = 6371.0

    # Distância
    distancia = R * c
    return distancia

df_train_merged['distance_customer_terminal'] = df_train_merged.apply(
    lambda row: haversine(row['x_customer_id'], row['y_customer_id'], row['x_terminal_id'], row['y_terminal_id']),
    axis=1
)

df_test_merged['distance_customer_terminal'] = df_test_merged.apply(
    lambda row: haversine(row['x_customer_id'], row['y_customer_id'], row['x_terminal_id'], row['y_terminal_id']),
    axis=1
)

In [ ]:
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,month_reference,PERIODO_DIA,hour_of_day,day_of_week,day_of_month,month,year,is_weekend,time_diff_transactions,distance_customer_terminal
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,202108,noite,3,6,1,8,2021,True,NaT,157.692238
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,202108,manhã,5,6,1,8,2021,True,0 days 02:29:38,2762.732805
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,202108,manhã,10,6,1,8,2021,True,0 days 05:09:34,920.672509
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,202108,tarde,15,6,1,8,2021,True,0 days 04:58:13,145.190544
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,202108,noite,19,6,1,8,2021,True,0 days 03:47:05,2593.697073


In [ ]:
df_train_merged[['CUSTOMER_ID','TERMINAL_ID','x_customer_id','y_customer_id','x_terminal_id','y_terminal_id','distance_customer_terminal']]

,CUSTOMER_ID,TERMINAL_ID,x_customer_id,y_customer_id,x_terminal_id,y_terminal_id,distance_customer_terminal
69,0,1133,10.950170,59.768684,8.533587,59.061828,157.692238
209,0,1138,10.950170,59.768684,-6.145777,37.452541,2762.732805
749,0,1530,10.950170,59.768684,26.127946,57.292877,920.672509
1338,0,241,10.950170,59.768684,10.849377,61.073466,145.190544
1650,0,1536,10.950170,59.768684,-6.049117,39.053767,2593.697073
...,...,...,...,...,...,...,...
280706,999,753,18.215288,59.518248,16.778398,48.820969,1193.082459
280838,999,119,18.215288,59.518248,14.065171,45.762837,1554.199767
281189,999,753,18.215288,59.518248,16.778398,48.820969,1193.082459
286767,999,959,18.215288,59.518248,-8.110124,37.376444,3093.037094


## 2.4 Valor total transacionado nas últimas X horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Ordena o DataFrame por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])


# Loop para aplicar a função de soma de valores de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'TOTAL_AMOUNT_PER_CUSTOMER_LAST{time_window.upper()}'

    # Função para calcular a soma dos valores das transações nas últimas 'time_window' para cada cliente
    def calculate_total_amount_recent(transactions):
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].sum()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_total_amount_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_total_amount_recent).reset_index(level=0, drop=True)

# Exibe as primeiras linhas do dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_AMOUNT_PER_CUSTOMER_LAST8H,TOTAL_AMOUNT_PER_CUSTOMER_LAST12H,TOTAL_AMOUNT_PER_CUSTOMER_LAST24H,TOTAL_AMOUNT_PER_CUSTOMER_LAST48H,TOTAL_AMOUNT_PER_CUSTOMER_LAST72H,TOTAL_AMOUNT_PER_CUSTOMER_LAST7D,TOTAL_AMOUNT_PER_CUSTOMER_LAST14D,TOTAL_AMOUNT_PER_CUSTOMER_LAST21D,TOTAL_AMOUNT_PER_CUSTOMER_LAST30D,TOTAL_AMOUNT_PER_CUSTOMER_LAST45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,191.12,191.12,191.12,191.12,191.12,191.12,191.12,191.12,191.12,191.12
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,287.62,287.62,287.62,287.62,287.62,287.62,287.62,287.62,287.62,287.62
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,178.69,308.30,369.81,369.81,369.81,369.81,369.81,369.81,369.81,369.81
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,132.96,229.46,420.58,420.58,420.58,420.58,420.58,420.58,420.58,420.58


## 2.5 Média de valor transacionado nas últimas X horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Ordena o DataFrame por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Loop para aplicar a função de soma de valores de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'MEAN_AMOUNT_PER_CUSTOMER_LAST{time_window.upper()}'

    # Função para calcular a soma dos valores das transações nas últimas 'time_window' para cada cliente
    def calculate_mean_amount_recent(transactions):
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].mean()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_mean_amount_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_mean_amount_recent).reset_index(level=0, drop=True)

# Exibe as primeiras linhas do dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,MEAN_AMOUNT_PER_CUSTOMER_LAST8H,MEAN_AMOUNT_PER_CUSTOMER_LAST12H,MEAN_AMOUNT_PER_CUSTOMER_LAST24H,MEAN_AMOUNT_PER_CUSTOMER_LAST48H,MEAN_AMOUNT_PER_CUSTOMER_LAST72H,MEAN_AMOUNT_PER_CUSTOMER_LAST7D,MEAN_AMOUNT_PER_CUSTOMER_LAST14D,MEAN_AMOUNT_PER_CUSTOMER_LAST21D,MEAN_AMOUNT_PER_CUSTOMER_LAST30D,MEAN_AMOUNT_PER_CUSTOMER_LAST45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,61.510000,61.510000,61.510000,61.510000,61.510000,61.510000,61.510000,61.510000,61.510000,61.510000
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,95.560000,95.560000,95.560000,95.560000,95.560000,95.560000,95.560000,95.560000,95.560000,95.560000
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,95.873333,95.873333,95.873333,95.873333,95.873333,95.873333,95.873333,95.873333,95.873333,95.873333
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,89.345000,102.766667,92.452500,92.452500,92.452500,92.452500,92.452500,92.452500,92.452500,92.452500
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,66.480000,76.486667,84.116000,84.116000,84.116000,84.116000,84.116000,84.116000,84.116000,84.116000


## 2.6 Mediana de valor transacionado nas últimas X horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Ordena o DataFrame por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Loop para aplicar a função de soma de valores de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'MEDIAN_AMOUNT_PER_CUSTOMER_LAST{time_window.upper()}'

    # Função para calcular a soma dos valores das transações nas últimas 'time_window' para cada cliente
    def calculate_median_amount_recent(transactions):
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].median()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_median_amount_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_median_amount_recent).reset_index(level=0, drop=True)

# Exibe as primeiras linhas do dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,MEDIAN_AMOUNT_PER_CUSTOMER_LAST8H,MEDIAN_AMOUNT_PER_CUSTOMER_LAST12H,MEDIAN_AMOUNT_PER_CUSTOMER_LAST24H,MEDIAN_AMOUNT_PER_CUSTOMER_LAST48H,MEDIAN_AMOUNT_PER_CUSTOMER_LAST72H,MEDIAN_AMOUNT_PER_CUSTOMER_LAST7D,MEDIAN_AMOUNT_PER_CUSTOMER_LAST14D,MEDIAN_AMOUNT_PER_CUSTOMER_LAST21D,MEDIAN_AMOUNT_PER_CUSTOMER_LAST30D,MEDIAN_AMOUNT_PER_CUSTOMER_LAST45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,61.510,61.51,61.510,61.510,61.510,61.510,61.510,61.510,61.510,61.510
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,95.560,95.56,95.560,95.560,95.560,95.560,95.560,95.560,95.560,95.560
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,96.500,96.50,96.500,96.500,96.500,96.500,96.500,96.500,96.500,96.500
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,89.345,96.50,89.345,89.345,89.345,89.345,89.345,89.345,89.345,89.345
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,66.480,82.19,82.190,82.190,82.190,82.190,82.190,82.190,82.190,82.190


## 2.7 Mínimo valor transacionado nas últimas X horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Ordena o DataFrame por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Loop para aplicar a função de soma de valores de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'MIN_AMOUNT_PER_CUSTOMER_LAST{time_window.upper()}'

    # Função para calcular a soma dos valores das transações nas últimas 'time_window' para cada cliente
    def calculate_min_amount_recent(transactions):
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].min()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_min_amount_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_min_amount_recent).reset_index(level=0, drop=True)

# Exibe as primeiras linhas do dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,MIN_AMOUNT_PER_CUSTOMER_LAST8H,MIN_AMOUNT_PER_CUSTOMER_LAST12H,MIN_AMOUNT_PER_CUSTOMER_LAST24H,MIN_AMOUNT_PER_CUSTOMER_LAST48H,MIN_AMOUNT_PER_CUSTOMER_LAST72H,MIN_AMOUNT_PER_CUSTOMER_LAST7D,MIN_AMOUNT_PER_CUSTOMER_LAST14D,MIN_AMOUNT_PER_CUSTOMER_LAST21D,MIN_AMOUNT_PER_CUSTOMER_LAST30D,MIN_AMOUNT_PER_CUSTOMER_LAST45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,82.19,82.19,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,50.77,50.77,50.77,50.77,50.77,50.77,50.77,50.77,50.77,50.77


## 2.8 Máximo valor transacionado nas últimas X horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Ordena o DataFrame por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Loop para aplicar a função de soma de valores de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'MAX_AMOUNT_PER_CUSTOMER_LAST{time_window.upper()}'

    # Função para calcular a soma dos valores das transações nas últimas 'time_window' para cada cliente
    def calculate_max_amount_recent(transactions):
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].max()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_max_amount_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_max_amount_recent).reset_index(level=0, drop=True)

# Exibe as primeiras linhas do dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,MAX_AMOUNT_PER_CUSTOMER_LAST8H,MAX_AMOUNT_PER_CUSTOMER_LAST12H,MAX_AMOUNT_PER_CUSTOMER_LAST24H,MAX_AMOUNT_PER_CUSTOMER_LAST48H,MAX_AMOUNT_PER_CUSTOMER_LAST72H,MAX_AMOUNT_PER_CUSTOMER_LAST7D,MAX_AMOUNT_PER_CUSTOMER_LAST14D,MAX_AMOUNT_PER_CUSTOMER_LAST21D,MAX_AMOUNT_PER_CUSTOMER_LAST30D,MAX_AMOUNT_PER_CUSTOMER_LAST45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,96.50,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,82.19,96.50,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61


## 2.9 Quantidade total de transações realizadas pelo cliente nas últimas X horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Loop para aplicar a função de contagem de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST{time_window.upper()}'

    # Função para calcular a quantidade de transações nas últimas 'time_window' horas para cada cliente
    def calculate_transaction_count_recent(transactions):
        # Contando apenas as transações em uma janela de 'time_window'
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].count()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_transaction_count_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_transaction_count_recent).reset_index(level=0, drop=True)

# Exibir o dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST8H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST12H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST24H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST48H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST72H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST7D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,2.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,2.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


## 2.10 Tempo desde a última transação

In [ ]:
# Calcula o tempo desde a última transação
df_train_merged['days_since_last_transaction'] = df_train_merged.groupby('CUSTOMER_ID')['TX_DATETIME'].diff().dt.days
df_test_merged['days_since_last_transaction'] = df_test_merged.groupby('CUSTOMER_ID')['TX_DATETIME'].diff().dt.days

df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST12H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST24H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST48H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST72H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST7D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST45D,days_since_last_transaction
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,0.0
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0


## 2.11 Flag para transações consecutivas no mesmo terminal

In [ ]:
# Flag para transações consecutivas no mesmo terminal
df_train_merged['consecutive_transactions_same_terminal'] = (df_train_merged['TERMINAL_ID'] == df_train_merged.groupby('CUSTOMER_ID')['TERMINAL_ID'].shift(1)).astype(int)
df_test_merged['consecutive_transactions_same_terminal'] = (df_test_merged['TERMINAL_ID'] == df_test_merged.groupby('CUSTOMER_ID')['TERMINAL_ID'].shift(1)).astype(int)

df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST24H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST48H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST72H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST7D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST45D,days_since_last_transaction,consecutive_transactions_same_terminal
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,0
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,0.0,0
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0


## 2.12 Proporção de transações em períodos de alto risco

In [ ]:
# Definindo um período de risco, como "noite"
high_risk_periods = ['noite']

# Ordena os DataFrames por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Função para calcular a proporção acumulada de transações em períodos de alto risco
def calculate_high_risk_ratio(transactions):
    # Converte os valores para 1 se estiverem no período de risco, caso contrário, 0
    binary_risk = transactions.isin(high_risk_periods).astype(int)
    # Calcula a média acumulativa
    return binary_risk.expanding().mean()

# Aplica a função de proporção acumulada no contexto do groupby para cada CUSTOMER_ID
df_train_merged['transaction_ratio_high_risk_period'] = df_train_merged.groupby('CUSTOMER_ID')['PERIODO_DIA'].apply(calculate_high_risk_ratio).reset_index(level=0, drop=True)
df_test_merged['transaction_ratio_high_risk_period'] = df_test_merged.groupby('CUSTOMER_ID')['PERIODO_DIA'].apply(calculate_high_risk_ratio).reset_index(level=0, drop=True)

# Exibir o DataFrame resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST48H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST72H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST7D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST45D,days_since_last_transaction,consecutive_transactions_same_terminal,transaction_ratio_high_risk_period
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,0,1.000000
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0,0.500000
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0,0.333333
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,0.0,0,0.250000
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0,0.400000


## 2.13 Flag para indicar transação acima da média de valor transacionado pelo cliente

In [ ]:
# Ordena os DataFrames por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Função para calcular a média cumulativa de transações até o momento da transação atual
def calculate_cumulative_mean(transactions):
    return transactions.expanding().mean()

# Aplica a função para calcular a média cumulativa por cliente no conjunto de treino
df_train_merged['cumulative_avg_amount'] = df_train_merged.groupby('CUSTOMER_ID')['TX_AMOUNT'].apply(calculate_cumulative_mean).reset_index(level=0, drop=True)

# Aplica a função para calcular a média cumulativa por cliente no conjunto de teste
df_test_merged['cumulative_avg_amount'] = df_test_merged.groupby('CUSTOMER_ID')['TX_AMOUNT'].apply(calculate_cumulative_mean).reset_index(level=0, drop=True)

# Define o alerta comparando o valor da transação com a média cumulativa
df_train_merged['high_value_alert'] = (df_train_merged['TX_AMOUNT'] > df_train_merged['cumulative_avg_amount']).astype(int)
df_test_merged['high_value_alert'] = (df_test_merged['TX_AMOUNT'] > df_test_merged['cumulative_avg_amount']).astype(int)

# Visualizar o resultado
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST7D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST45D,days_since_last_transaction,consecutive_transactions_same_terminal,transaction_ratio_high_risk_period,cumulative_avg_amount,high_value_alert
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,1.0,NaN,0,1.000000,61.510000,0
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,2.0,0.0,0,0.500000,95.560000,1
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,3.0,0.0,0,0.333333,95.873333,1
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,4.0,4.0,4.0,4.0,4.0,0.0,0,0.250000,92.452500,0
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,5.0,5.0,5.0,5.0,5.0,0.0,0,0.400000,84.116000,0


## 2.14 Contagem de terminais distintos utilizados pelo cliente

In [ ]:
import pandas as pd

# Ordena os DataFrames por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Função para calcular o número total de terminais distintos usados até a transação atual
def calculate_unique_terminals_count(transactions):
    unique_terminals = set()
    unique_terminal_counts = []

    for terminal in transactions:
        unique_terminals.add(terminal)  # Adiciona o terminal ao conjunto de únicos
        unique_terminal_counts.append(len(unique_terminals))  # Adiciona a contagem total de terminais únicos

    return pd.Series(unique_terminal_counts, index=transactions.index)

# Aplica a função para calcular o número total de terminais distintos usados até cada transação no conjunto de treino
df_train_merged['distinct_terminals_count'] = df_train_merged.groupby('CUSTOMER_ID')['TERMINAL_ID'].apply(calculate_unique_terminals_count).reset_index(level=0, drop=True)

# Aplica a função para calcular o número total de terminais distintos usados até cada transação no conjunto de teste
df_test_merged['distinct_terminals_count'] = df_test_merged.groupby('CUSTOMER_ID')['TERMINAL_ID'].apply(calculate_unique_terminals_count).reset_index(level=0, drop=True)

# Visualizar o resultado
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST45D,days_since_last_transaction,consecutive_transactions_same_terminal,transaction_ratio_high_risk_period,cumulative_avg_amount,high_value_alert,distinct_terminals_count
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,NaN,0,1.000000,61.510000,0,1
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,0.0,0,0.500000,95.560000,1,2
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,0.0,0,0.333333,95.873333,1,3
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,4.0,4.0,4.0,4.0,0.0,0,0.250000,92.452500,0,4
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,5.0,5.0,5.0,5.0,0.0,0,0.400000,84.116000,0,5


## 2.15 Razões entre features

In [ ]:
# Razão entre o valor médio transacionado em 1H pelo valor médio transacionado em XH ou X Dias
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST1H_TO_2H'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST2H']
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST1H_TO_4H'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST4H']
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST1H_TO_8H'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST8H']
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST1H_TO_24H'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST24H']
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST1H_TO_7D'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST7D']
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST1H_TO_14D'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST14D']

# Razão entre o valor médio transacionado em 1H pelo valor médio transacionado em XH ou X Dias
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H_TO_2H'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST2H']
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H_TO_4H'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST4H']
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H_TO_8H'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST8H']
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H_TO_24H'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST24H']
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H_TO_7D'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST7D']
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H_TO_14D'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST14D']

In [ ]:
# Exportando o DataFrame para um arquivo CSV
df_train_merged.to_csv('/content/drive/MyDrive/Prevencao_fraude/df_train_full.csv', index=False)
df_test_merged.to_csv('/content/drive/MyDrive/Prevencao_fraude/df_test_full.csv', index=False)